In [ ]:
import pennylane as qml
from pennylane import numpy as np

## N.2.1a

In [ ]:
def apply_gate_mixed(rho,U):
    
    """
    Args:
        rho: (np.array(array[complex]): The density matrix of the input state
        U (np.array(array[complex])): A matrix representing the unitary gate U to be applied.
        
    Returns:
        (np.array([array[complex]])): The density matrix of the output state.
    """

    ################
    #YOUR CODE HERE#
    ################
    
    return U @ rho @ np.conj(U).T # Return the density matrix

U = qml.matrix(qml.RX(np.pi/3,0))
rho = np.array([[3/4,1/4],[1/4,1/4]])

print("A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:")
print(apply_gate_mixed(rho,U).round(2))


## N.2.1b

In [ ]:
dev = qml.device('default.mixed', wires=1) # Define a default.mixed device for 1 qubit

@qml.qnode(dev)
def apply_gate_circuit(rho,U):

    """
    Args:
        rho: (np.array(array[complex]): The density matrix of the input state.
        U (np.array(array[complex])): A matrix representing the unitary gate U to be applied.
        
    Returns:
        (np.array([array[complex]])): The density matrix of the output state.
    """
    
    # Prepare a state with density operator rho
    qml.QubitDensityMatrix(rho, wires=0)
    # Apply the unitary U
    qml.QubitUnitary(U, wires=0)
    
    return qml.state() # Return the density operator after applying U

U = qml.matrix(qml.RX(np.pi/3,0))
rho = np.array([[3/4,1/4],[1/4,1/4]])

print("A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:")
print(apply_gate_circuit(rho,U).round(2))


## N.2.2a

In [ ]:
def eigenprojectors(obs):
    
    """
    Args:
        obs (np.array([array[complex]])): A Hermitian operator representing a quantum observable.
        
    Returns:
        (np.array(array[array[complex]])): An array containing the eigenprojectors of the observable.
        Therefore, it is an array that contains matrices.
    """

    ################
    #YOUR CODE HERE#
    ################

    eigenvectors = # Use numpy's linalg to calculate the eigenvectors of obs
    
    projectors = # Use list comprehension to build a list of eigenprojectors from the eigenvectors
    
    return projectors
